In [9]:
#Importing the basis libraries
import numpy as np
import pandas as pd

#Importing the cv and Keras for model training
import cv2
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Convolution2D,Dropout,MaxPool2D,BatchNormalization,Flatten
from keras.optimizers import adam,RMSprop
from keras.callbacks import ReduceLROnPlateau,EarlyStopping
import tensorflow as tf
from keras.utils import to_categorical

#Loading the data from Kaggle facial emotion recognization competition 2013
# (https://github.com/atulapra/Emotion-detection/blob/master/Tensorflow/emotions.py)
#Data has three columns (Emotion Label,Usage whether training or test,pixel values)

train_data=pd.read_csv('data/train.csv')
y=train_data['emotion'] 
X=[i.split(' ') for i in train_data['pixels']] #pixel values are seperated by space 48*48=2304

#-------------------------------------------------------------
num_classes = 7 # 0 to 6 as angry, disgust, fear, happy, sad and surprise, neutral
batch_size = 256
epochs = 30
#-------------------------------------------------------------


In [10]:
x_train = np.array(X, 'float32') #X is list, so converted to array
y_train = y.astype('float32')

In [11]:
x_train /= 255 #normalizing pixel values between 0 and 1
y_train=to_categorical(y_train,num_classes) #conveting categorical varible

In [16]:
x_train=x_train.reshape(-1,48,48,1)

In [21]:
x_train.shape

(28709, 48, 48, 1)

In [22]:
#Model
model=Sequential()

In [23]:
#layer1
model.add(Convolution2D(32,(4,4),padding='same',input_shape=(48,48,1)))
model.add(Convolution2D(64,(4,4),padding='same'))
model.add(MaxPool2D())
model.add(Dropout(0.3))
#layer2
model.add(Convolution2D(64,(4,4),padding='same'))
model.add(MaxPool2D())
model.add(Dropout(0.3))
#layer3
model.add(Convolution2D(128,(4,4),padding='same'))
model.add(MaxPool2D())
model.add(Convolution2D(128,(4,4),padding='same'))
model.add(MaxPool2D())
model.add(Dropout(0.3))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes, activation='softmax'))


In [24]:
ada=adam()

In [25]:
#------------------------------
#batch process
gen = ImageDataGenerator()
train_generator = gen.flow(x_train, y_train, batch_size=batch_size)

#------------------------------

model.compile(loss='categorical_crossentropy', optimizer=ada, metrics=['accuracy'])

In [26]:
model.fit_generator(train_generator, steps_per_epoch=batch_size, epochs=epochs) #train for randomly selected one
#model.load_weights('/data/facial_expression_model_weights.h5') #load weights

Epoch 1/30
256/256 [==============================] - 348s 1s/step - loss: 1.6068 - accuracy: 0.3613
Epoch 2/30
256/256 [==============================] - 349s 1s/step - loss: 1.2372 - accuracy: 0.5265
Epoch 3/30
256/256 [==============================] - 346s 1s/step - loss: 1.0698 - accuracy: 0.5945
Epoch 4/30
256/256 [==============================] - 349s 1s/step - loss: 0.9164 - accuracy: 0.6534
Epoch 5/30
256/256 [==============================] - 347s 1s/step - loss: 0.7636 - accuracy: 0.7129
Epoch 6/30
256/256 [==============================] - 1196s 5s/step - loss: 0.6567 - accuracy: 0.7560
Epoch 7/30
256/256 [==============================] - 350s 1s/step - loss: 0.5703 - accuracy: 0.7893
Epoch 8/30
256/256 [==============================] - 351s 1s/step - loss: 0.4853 - accuracy: 0.8230
Epoch 9/30
256/256 [==============================] - 349s 1s/step - loss: 0.4275 - accuracy: 0.8449
Epoch 10/30
256/256 [==============================] - 351s 1s/step - loss: 0.3795 - accur

In [28]:
model.save_weights('data/facial_expression_detection_model.h5')